In [ ]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/2020.deeplearning/master/init.py
from init import init; init(force_download=False)

In [ ]:
try:
    %tensorflow_version 2.x
    print ("Using TF2 in Google Colab")
except:
    pass
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext tensorboard

from sklearn.datasets import *
from local.lib import mlutils
tf.__version__



**A multilayer perceptron**

assuming $n$ layers, the output at layer $i$

$$\mathbf{a}_i = \text{activation}(\mathbf{a}_{i-1} \cdot \mathbf{W}_i + \mathbf{b}_i)$$

at the first layer

$$\mathbf{a}_0 = \text{activation}(\mathbf{X} \cdot \mathbf{W}_0 + \mathbf{b}_0)$$

and the layer prediction is the output of the last layer:

$$\hat{\mathbf{y}} = \mathbf{a}_{n-1}$$ 

with $\text{activation}$ being an activation function, such as $\text{sigmoid}(z) = \frac{1}{1+e^{-z}}$, $\text{tanh}$, $\text{ReLU}$, etc.


**Cost (with regularization)**


$$J(\mathbf{b}_1, b_2, \mathbf{W}_1, \mathbf{W}_2) = \frac{1}{m}\sum_{i=0}^{m-1} (\hat{y}-y)^2 + \lambda \sum_{i=0}^{n-1} \bigg[ \| \mathbf{b}_i\|^2 + \|\mathbf{W}_i\|^2 \bigg]$$


$\lambda$ regulates the participation of the regularization terms. Given a vector or matrix $\mathbf{T}$, its squared norm is denoted by $||\mathbf{T}||^2 \in \mathbb{R}$ and it's computed by squaring all its elements and summing them all up. 


## Task 1: Multilayer perceptron model with low level API

build a custom model (`build`, `call`) using standard optimization (`model.compile`, etc.)

your class must:

- accept in the constructor:
   - `neurons`: a list containing the number of neurons of each layer
   - `activations`: a list of strings containing one of `'sigmoid'`, `'tanh'`, `'linear'`, `'relu'` so that the corresponding `tf.keras.activations` function is used.

- **include Tensorboard callbacks for BOTH LOSS AND ACCURACY. See the [example here](https://www.tensorflow.org/tensorboard/get_started)**. You will have to add the appropriate [keras metric](https://www.tensorflow.org/api_docs/python/tf/keras/metrics) according to your model output.

- create a **custom loss function** to include the regularization parameter and **use it when compiling the model**. Look in the internet for tutorials, for example [here](https://towardsdatascience.com/advanced-keras-constructing-complex-custom-losses-and-metrics-c07ca130a618), and many others

for instance, the following code:

    mlp = MLP(neurons=[10,1], activations=["tanh","sigmoid"])

must create a MLP with two layers with 10 and 1 neurons each, and `tanh` and `sigmoid` activation correspondingly.


Observe that:

- as you are following the Keras class API (`call`+ `build`) your model should work as any other model (`compile`, `fit`, etc.).

- the `loss` method of your MLP instance must be passed on when calling `compile`



In [ ]:
from tensorflow.keras.activations import relu, sigmoid, tanh, linear

In [ ]:
from progressbar import progressbar as pbar
from tensorflow.keras import Model
from tensorflow.keras.activations import relu, sigmoid, tanh, linear

class MLP(Model):
    def __init__(self, neurons, activations, reg=0.):
        super(MLP, self).__init__()
        assert len(neurons)==len(activations), "must have the same number of neurons and activations"
        
        ... # YOUR CODE HERE
        
    def build(self, input_shape):
        
        ... # YOUR CODE HERE
        
    @tf.function
    def call(self, X):
        ... # YOUR CODE HERE

    @tf.function
    def loss(self, y_true, y_pred):
        ... # YOUR CODE HERE


In [ ]:
X, y = make_moons(200, noise=.35)
X, y = X.astype(np.float32), y.astype(np.float32).reshape(-1,1)
plt.scatter(X[:,0][y[:,0]==0], X[:,1][y[:,0]==0], color="red", label="class 0")
plt.scatter(X[:,0][y[:,0]==1], X[:,1][y[:,0]==1], color="blue", label="class 1")

In [ ]:
!rm -rf logs
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs/no_regularization")

mlp = MLP(neurons=[10,1], activations=["tanh","sigmoid"])

mlp.compile(... # YOUR CODE HERE)

In [ ]:
mlp.fit(X,y, epochs=400, batch_size=16, verbose=0, 
        callbacks=[tensorboard_callback])

In [ ]:
predict = lambda X: (mlp.predict(X)[:,0]>0.5).astype(int)
mlutils.plot_2Ddata_with_boundary(predict, X, y.reshape(-1));
plt.title("accuracy %.2f"%np.mean(predict(X)==y.reshape(-1)));

regularization must work!!!

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs/with_regularization")
mlp = MLP(neurons=[10,1], activations=["tanh","sigmoid"], reg=0.001)

mlp.compile(... # YOUR CODE HERE)

mlp.fit(X,y, epochs=400, batch_size=10, verbose=0, callbacks=[tensorboard_callback])
mlutils.plot_2Ddata_with_boundary(predict, X, y.reshape(-1))
plt.title("accuracy %.2f"%np.mean(predict(X)==y.reshape(-1)));

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs


## Task 2: Multilayer perceptron model AND optimization loop with low level API

build a custom model such as in the exercise above and implement your optimization loop in `.fit`
 
observe that you will have to:
- use whichever method you choose from the corresponding notebook (custom SGD, `apply_gradients`, `train_on_batch`)
- write by hand loss and accuracy to Tensorboard. See  how[`tf.summary`](https://www.tensorflow.org/api_docs/python/tf/summary) works


In [ ]:

class MLP(Model):
    def __init__(self, neurons, activations, reg=0.):
        super(MLP, self).__init__()
        assert len(neurons)==len(activations), "must have the same number of neurons and activations"
        
        ... # YOUR CODE HERE
        
    def build(self, input_shape):
        
        ... # YOUR CODE HERE

                
    @tf.function
    def call(self, X):
        ... # YOUR CODE HERE


    @tf.function
    def loss(self, y_true, y_pred):
        ... # YOUR CODE HERE
    
    def fit(self, X, y, epochs, batch_size):
        ... # YOUR CODE HERE


In [ ]:

... use tensorboard!!! ...

mlp = MLP(neurons=[10,1], activations=["tanh","sigmoid"])
mlp.compile(... # YOUR CODE HERE)

mlp.fit(X,y, epochs=400, batch_size=16)

In [ ]:
mlutils.plot_2Ddata_with_boundary(predict, X, y.reshape(-1));
plt.title("accuracy %.2f"%np.mean(predict(X)==y.reshape(-1)));

regularization must work!!!

In [ ]:
... use tensorboard!!! ...

mlp = MLP(neurons=[10,1], activations=["tanh","sigmoid"], reg=0.01)
mlp.compile(... # YOUR CODE HERE)

mlp.fit(X,y, epochs=400, batch_size=16)

In [ ]:
mlutils.plot_2Ddata_with_boundary(predict, X, y.reshape(-1));
plt.title("accuracy %.2f"%np.mean(predict(X)==y.reshape(-1)));

In [ ]:
%tensorboard --logdir logs
